In [1]:
!pip install numpy pandas matplotlib

In [2]:
# Load environment variables

import dotenv
dotenv.load_dotenv("/Users/kennedyjayasumana/Downloads/env", override=True)

import openai
import os
import json
import re


In [3]:
instruction = """Generate a Markdown table with the title "Combotext Training Suggestions"
markdown
---
- **Phone Numbers**: contact phone numbers
- **Banking Identifiers**: Routing numbers, Swift codes, NMLS #, license numbers
- **Copyrights and Ownership Statements**:  full copyright string, any statements like (bank) is a wholly owned subsidiary of (owner)
- **Taglines and unique text**:  taglines, mobile app names, subsidiaries etc
- **Other Unique Content**:  examples include "Jedi Web Development", "jediWebDevelopement".
---



prompt:
Here's text extracted from 360 federal credit union's website for analysis and extraction: """

instruction = """Generate only a CSV separated as a output, where <what it is> might be a phone number, Banking Identifiers: Routing numbers, Swift codes, NMLS #, license numbers,Copyrights  Ownership Statements:  full copyright string, any statements like (bank) is a wholly owned subsidiary of (owner), or which subsidiaries they own, taglines and unique text, or other unique content as judged by you, please be light on the commentary and only include it when "what it is" doesnt adequately cover
<term/word/phrase>,<what it is>,<commentary>

ie 
"123-456-7878","phone number",""
"1131191144", "license number","plumber license"
"""

In [4]:
def display_response(question, answer):
    """Prints the question and the model's answer in a formatted manner."""
    print(f"Question: {question}")
    print(f"Answer: {answer}\n")

def save_response(file,question, answer):
    """Saves answer to /tmp/file"""
    file_path = f"/tmp/gpt1/{file}"
    with open(file_path, 'w') as f:
        f.write(f"{answer}")
         

def get_content_from_directory(directory_path):
    """Reads the contents of JSON files in a given directory and returns the value of 'content/BODY_TEXT[0]' key as a list."""
    content_list = []
    
    # List all files in the directory
    file_names = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]
    
    for file_name in file_names:
        with open(os.path.join(directory_path, file_name), 'r') as file:
            try:
                # Load the JSON content of the file
                data = json.load(file)
                
                # Check if the desired key exists and append its value to content_list
                if "content" in data and "BODY_TEXT" in data["content"] and len(data["content"]["BODY_TEXT"]) > 0:
                    content_list.append((file_name,data["content"]["BODY_TEXT"][0]["content"]))
            except Exception as e:
                print(f"Exception encountered while processing {file_name}: {e}")
    
    return content_list

def get_prompts_from_directory(filepath):
    # Read the contents of the file
    with open(filepath, 'r') as file:
        content = file.read()
    
    # Regular expression to match the pattern (e.g., "0. Josh:")
    pattern = r'(\d+\.\s*[^:]+):\s*(.*?)\s*(?=\d+\.\s*[^:]+:|$)'
    
    # Use the findall method to get all matches
    matches = re.findall(pattern, content, re.DOTALL)
    
    # Convert the matches to a list of pairs with title and content
    result = [(match[0].strip(), match[1].strip()) for match in matches]
    
    return result

In [5]:

         
def query(messages):
    chatbot_response = openai.ChatCompletion.create(model="gpt-4",messages=messages )
    return chatbot_response

content_list = get_content_from_directory("/tmp/scrapes")
#print(content_list[0])
prompt_list = get_prompts_from_directory("/tmp/contexts")
print(len(prompt_list))
#print(prompt_list[0])
i=0;

for content in content_list[0:10]:
    print("querying",i)
    i=i+1
    for persona in prompt_list:
        file_out=prompt[0]+content[0]
        file_out=file_out = re.sub(r'[^a-zA-Z0-9]', '', file_out)+".txt"
        
        message = [
            {"role": "system", "content": persona[1]},
            {"role": "user", "content": instruction +"\n here is the content:" + content[1]},
        ]
        print("query?",message)
        chatbot_response = query(message)
        chat_content_response=chatbot_response.choices[0].message['content']
        display_response(prompt[1],chat_content_response)
        save_response(file_out,prompt[1],chat_content_response)
        
    #chatbot_response = query(messages)
    #display_response( content, chatbot_response.choices[0].message['content'])

Exception encountered while processing .DS_Store: 'utf-8' codec can't decode byte 0x80 in position 3131: invalid start byte
17
querying 0
query? [{'role': 'system', 'content': "context: we are working together to extract Unique Words, Phrases, numbers and other text strings from web pages. We're trying to find a set of things that are unique enough that they can help us search for other web pages that are pretending to be the company whose page will be provided for analysis.  Unique numbers, especially phone numbers, routing numbers, NMLS IDs and Swift codes will be be important to extract. Unique text strings, such as copyright notices and marketing tag lines, will also be important. However, when selecting words, phrases, numbers or other text strings to extract, we want to make sure that the presence of one or more of these words identified on a potentially fraudulent page won't cause too many things to look at."}, {'role': 'user', 'content': 'Generate only a CSV separated as a outp

APIError: HTTP code 502 from API (<html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
<hr><center>cloudflare</center>
</body>
</html>
)